# $\mathscr I$ space
Space that models the information packet associated with a protein

In [33]:
class Information:
    def __init__(self, tupla):
        self.tupla = tupla
        
    def print():
        print(self.tupla)
        
    def vectorize():
        return np.array(self.tupla)   

# $\mathscr M$ set
OOP description of the set of possible mutations

In [34]:
class Mutation:
    def __init__(self, descriptor):
        self.descriptor = descriptor

# $\mathscr P$ space
OOP description of the space of proteins
<center><h1><b>Content (functions acting on $\mathscr P$)</b></h1></center>
<ul>
    <li><code>compute_q_value</code> : $\mathscr P \rightarrow \mathbb R_+$</li>
    computes the <i>q value</i> of the protein, performing the simulation. The result is then stored into the variable         <code>q_value</code>
    <li><code>mutate</code> : $\mathscr P \times \mathscr M \rightarrow \mathscr P$</li>
    produce a new protein, starting from the current one and the mutation requested
    <li><code>information_neighbors</code> : $\mathscr p \mapsto \{ (\mathscr q,I_{\mathscr q}) \}_{\mathscr q \in \mathcal N(\mathscr p)}$ </li>
    returns a mapping that associates each neighbor on the <i>protein lattice</i> with an information packet in the set $\mathscr I$
</ul>

In [36]:
class Protein:
    def __init__(self):
        self.q_value = None        # unitialized q value
        self.possible_mutations = [
            Mutation(0) , # 1 mutation
            Mutation(1)   # 2 mutation
            # ...
        ]
        # set of available_mutations
    def mutate(self,param):
        # returns a copy of the current protein 
        # with the requested mutation
        return self
    def information_neighbors(self):
        # returns a dictionary that associate each neighbors of the current protein 
        # in the mutation lattice with an information packet
        # e.g.
        #          Protein2J47.information_neighbors() ==> { 'mutation_1' : (I^1_1,I^1_2,...), 'mutation_2' : (...),.. }
        # initialize the dictionary
        ret = {}
                                    ##########################
        # populate the dictionary   #  TODO TODO TODO TODO   # 
        # ...                       #  TODO TODO TODO TODO   #
                                    ##########################
        # return the dictionary
        return ret
    
    #########################################################################################################
    
    def compute_q_value(self):            # TODO 
        # initializes the q value
        self.q_value = np.random.randn()
        
    def get_q_value(self):
        return self.q_value

    #########################################################################################################

# Learner
Agent able to learn from experience

In [39]:
import numpy as np
class Learner:
    def __init__(self):
        self.X = np.array([])
        self.Y = np.array([])
    def learn(self,x,y):
        if len(self.X) == 0:
            # first entry
            self.X = (x * 1)[None,:]
            self.Y = y * 1
        else:
            self.X = np.r_[self.X,x[None,:]]
            self.Y = np.r_[self.Y,y]
    def predict(self,x):
        # prediction function...
        return self.X,self.Y

In [40]:
L = Learner()
L.learn(np.random.randn(2),1)
L.learn(np.random.randn(2),1)
L.learn(np.random.randn(2),2)
L.predict(0)

(array([[-0.15500318,  1.98816421],
        [ 0.10748642, -1.01707568],
        [ 0.42173699, -0.23829369]]),
 array([1, 1, 2]))

# Guide
Agent who drives the configuration exploration, modulating the trust wrt the Learner

In [41]:
class Guide:
    def __init__(self):
        self.eta = 1.
        
    def trust():
        self.eta *= 1.1          # if I build trust, the "confidence in the learner" increase
        
    def untrust():
        self.eta *= 0.9          # if I disappoint you, otherwise...

        
    def softmax(probabilityDistribution):
        # returns the homotopied probability distribution
        
        return np.exp(probabilityDistribution)** self.eta / (np.exp(probabilityDistribution) ** self.eta).sum()

    def choose(probabilityDistribution):
        # takes as input a probability distribution and "homotopies" it with the uniform distribution
        # according to its trust.
        # returns the extracted index
        
        P = softmax(probabilityDistribution)
        return np.random.choice(len(P),P)

# Team
Class that models the cooperative system between the learner and the guide

In [46]:
class Team:
    def __init__(self):
        self.G = Guide()
        self.L = Learner()
        
    def optimize(self,protein):
        protein.compute_q_value()                                                # compute the q value of the input protein
        for i in range(100):                                                     # loop
            information_neighbors    =  protein.information_neighbors()          # map each neighbour with an information packet
            probability_distribution =  self.L.predict(information_neighbors)    # returns a dictionary that associate each mutation with a probability distribution
            selected_mutation        =  self.G.choose(probability_distribution)  # select the next mutation according to the learner's opinion
            
            new_protein              =  protein.mutate(selected_mutation)        # performs the mutation
            new_protein.compute_q_value()                                        # compute the q value for the current mutation
            
            self.L.learn(                                                        # give to the Learner the knowledge about 
                    information_neighbors[selected_mutation],                    # the mapping between chosen mutation information packet
                    new_protein.get_q_value()                                    # and the q value associated to the chosen mutation
            )
            
            if( new_protein.get_q_value() <= protein.get_q_value() ):            # greedy descent
                protein = new_protein                                            # since q value has been computed for new_protein it's stored in new_protein and therefore at this point in the protein
                self.G.trust()                                                   # builds trust between guide and learner
            else:
                self.G.untrust()                                                 # damages trust between guide and learner
            print( protein.get_q_value() )

In [47]:
import numpy as np
np.array((1,2,3))

array([1, 2, 3])